In [484]:
import pyomo.environ as pyomo

# solver
solver = pyomo.SolverFactory('cbc',executable=r'C:\Cbc-master-win64-msvc16-mt\bin\cbc.exe')
solver.options['sec'] = 5

In [485]:
import pandas as pd 
#df = pd.read_csv('two node example_multiple_d.csv') #multiple dd combi 
df = pd.read_csv('two node example_single_d.csv')
t_set = df['t'].unique()

cost_df = pd.read_csv("two node example_cost.csv")
# cost_df = pd.read_csv("two node example_cost_testset1st.csv")
# cost_df = pd.read_csv("two node example_cost_2ndtestset.csv")
s_set = cost_df['s'].unique()

In [486]:
import csv
import pyomo.environ as pyo

In [487]:
print(df) 
print(cost_df)


    s  t  d_1t  d_2t
0   1  1     1     1
1   1  2     5     5
2   1  3    10    10
3   2  1     1     1
4   2  2     5     5
5   2  3    20    20
6   3  1     1     1
7   3  2     5     5
8   3  3    30    30
9   4  1     1     1
10  4  2    10    10
11  4  3    20    20
12  5  1     1     1
13  5  2    10    10
14  5  3    10    10
15  6  1     1     1
16  6  2    10    10
17  6  3    30    30
   s  c_H_1_1  c_H_1_2  c_H_2_1  c_H_2_2  c_G_1_1  c_G_1_2  c_G_2_1  c_G_2_2  \
0  1        1        1        2        2        1        1        2        2   
1  2        1        2        3        4        1        2        3        4   
2  3        1        2        3        4        4        3        2        1   
3  4        1        2        2        1        1        1        1        1   
4  5        1        1        1        2        1        1        1        1   
5  6        3        3        3        3        3        3        1        3   

   c_F  
0    2  
1   50  
2    3  
3   

In [488]:
# constraints 

##  \sum_{k \in K_1} g_{1,k,t} - f_t \geq d_{1,t}~~\forall t \in T ~~~{\color{blue} \lambda_{1,t}}
def demand1(model, t):
    #if t == 3: 
    #     return (model.g_1_1_t[t] + model.g_1_2_t[t] - model.f_t[t] >= model.d_1t[t] + 1)
    #else: 
        return (model.g_1_1_t[t] + model.g_1_2_t[t] - model.f_t[t] >= model.d_1t[t])

#   \sum_{k \in K_2} g_{2,k,t} + f_t \geq d_{2,t}~~\forall t \in T,  ~~~{\color{blue} \lambda_{2,t}}
def demand2(model, t):
    # if t == 3: 
    #    return (model.g_2_1_t[t] + model.g_2_2_t[t]+ model.f_t[t] >= model.d_2t[t]+ 1 )
    #else: 
        return (model.g_2_1_t[t] + model.g_2_2_t[t]+ model.f_t[t] >= model.d_2t[t])

##  g_{i,k,t} -  h_{i,k} \leq 0 ~~\forall i, \forall t \in T~~~{\color{blue}\pi_{i,k,t}}
# i = 1, k = 1 
def generator_capacity_1_1 (model,t): 
    #if t == 3: 
    #     return (model.g_1_1_t [t] - model.h_1_1 <= 0 + 1)
    #else: 
         return (model.g_1_1_t [t] - model.h_1_1 <= 0) 
# i =2 , k = 1 
def generator_capacity_2_1 (model, t): 
    #if t == 3: 
    #     return (model.g_2_1_t[t] - model.h_2_1 <= 0 + 1)
    #else: 
         return (model.g_2_1_t[t] - model.h_2_1 <= 0) 
# i =1 , k =2 
def generator_capacity_1_2 (model, t): 
    #if t == 3: 
    #    return (model.g_1_2_t[t] - model.h_1_2 <= 0 + 1)
    #else: 
        return (model.g_1_2_t[t] - model.h_1_2 <= 0 )

# i = 2, k = 2 
def generator_capacity_2_2 (model, t): 
    #if t == 3: 
    #      return( model.g_2_2_t[t] - model.h_2_2 <= 0 + 1)
    #else: 
        return( model.g_2_2_t[t] - model.h_2_2 <= 0 )

## transmission capacity 
def transmission_capacity_positive (model, t): 
   #if t ==3: 
    #      return (model.f_t [t] - model.F <= 0 + 1 )          
   #else: 
         return (model.f_t [t] - model.F <= 0)

def transmission_capacity_negative (model, t): 
    if t == 3: 
        return( - model.f_t [t] - model.F <= 0 +1)
    else: 
        return( - model.f_t [t] - model.F <= 0)

In [489]:
output_file = "model_results.csv"
first_write = True 

In [490]:
dual_dict = dict()
for s in s_set :
# for s in range (1,7) :
    #if s == 1: # test a specific set 


        print("s=" , s)

        # create a model 
        model = pyomo.ConcreteModel()

        # series 
        model.nt = pyomo.Param(initialize=len(df))
        model.T = pyomo.Set(initialize=t_set)

        # c^H_{i,k} 
        model.c_H_1_1 = pyomo.Param(initialize = cost_df.loc[(cost_df['s'] == s) , 'c_H_1_1'].values[0]  ) # i = 1, k = 1
        model.c_H_1_2 = pyomo.Param(initialize = cost_df.loc[(cost_df['s'] == s) , 'c_H_1_2'].values[0]  ) # i = 2, k = 1
        model.c_H_2_1 = pyomo.Param(initialize = cost_df.loc[(cost_df['s'] == s) , 'c_H_2_1'].values[0]  ) # i = 1, k = 2
        model.c_H_2_2 = pyomo.Param(initialize = cost_df.loc[(cost_df['s'] == s) , 'c_H_2_2'].values[0]  ) # i = 2, k = 2
        
        model.c_G_1_1 = pyomo.Param(initialize = cost_df.loc[(cost_df['s'] == s) , 'c_G_1_1'].values[0]  ) # i = 1, k = 1
        model.c_G_1_2 = pyomo.Param(initialize = cost_df.loc[(cost_df['s'] == s) , 'c_G_1_2'].values[0]  ) # i = 2, k = 1
        model.c_G_2_1 = pyomo.Param(initialize = cost_df.loc[(cost_df['s'] == s) , 'c_G_2_1'].values[0]  ) # i = 1, k = 2
        model.c_G_2_2 = pyomo.Param(initialize = cost_df.loc[(cost_df['s'] == s) , 'c_G_2_2'].values[0]  ) # i = 2, k = 2
        
        model.c_F = pyomo.Param( initialize = cost_df.loc[(cost_df['s'] == s) , 'c_F'].values[0])
        #demand 
        model.d_1t = pyomo.Param(model.T, initialize = {t: df.loc[(df['t'] == t), 'd_1t'].values[0] for t in t_set} )        
        model.d_2t = pyomo.Param(model.T, initialize = {t: df.loc[(df['t'] == t), 'd_2t'].values[0] for t in t_set} )
        
        #model.d_1t = pyomo.Param(model.T, initialize = {t: df.loc[(df['s'] == s) & (df['t'] == t), 'd_1t'].values[0] for t in t_set} )
        #model.d_2t = pyomo.Param(model.T, initialize = {t: df.loc[(df['s'] == s) & (df['t'] == t), 'd_2t'].values[0] for t in t_set} )

        # Variables
        # g_{1,k,t} 
        model.g_1_1_t = pyomo.Var(model.T, domain = pyomo.NonNegativeReals) #g_{1,1,t}, k = 1 
        model.g_1_2_t = pyomo.Var (model.T, domain = pyomo.NonNegativeReals) #g_{1,2,t}, k = 2
        # g_{2,k,t} 
        model.g_2_1_t = pyomo.Var(model.T, domain = pyomo.NonNegativeReals) #g_{2,1,t}, k = 1 
        model.g_2_2_t = pyomo.Var (model.T, domain = pyomo.NonNegativeReals) #g_{2,2,t}, k = 2
        #h_{i,k} - node i, tech k 
        model.h_1_1 = pyomo.Var (domain = pyomo.NonNegativeReals) #h_{1,1}, i = 1, k = 1
        model.h_1_2 = pyomo.Var (domain = pyomo.NonNegativeReals) #h_{1,2}, i = 1, k = 2 
        model.h_2_1 = pyomo.Var (domain = pyomo.NonNegativeReals) #h_{1,1}, i = 2, k = 1
        model.h_2_2 = pyomo.Var (domain = pyomo.NonNegativeReals) #h_{1,2}, i = 2, k = 2 
        model.f_t = pyomo.Var(model.T, domain = pyomo.Reals) # +ve is node 1 to node 2, -ve is node 2 to node 1 
        model.F = pyomo.Var ( domain = pyomo.NonNegativeReals) 

        # objective function 
        #  \min ~ \sum_{i} \left( \sum_{k}  \left(  c^H_{i,k} h_{i,k} + \sum_{t} c^G_{i,k}g_{i,k,t} \right)\right) + c^F F
        def objective_func(model): 
            return (
                model.c_H_1_1 * model.h_1_1
                + model.c_H_1_2 * model.h_1_2
                + model.c_H_2_1 * model.h_2_1
                + model.c_H_2_2 * model.h_2_2
                + sum(model.c_G_1_1 * model.g_1_1_t[t] for t in model.T)
                + sum(model.c_G_1_2 * model.g_1_2_t[t] for t in model.T)
                + sum(model.c_G_2_1 * model.g_2_1_t[t] for t in model.T)
                + sum(model.c_G_2_2 * model.g_2_2_t[t] for t in model.T)
                + model.c_F * model.F
            )


        model.objective = pyomo.Objective(rule = objective_func, sense = pyomo.minimize)

        # Constraints 
        model.demand1Constraint = pyomo.Constraint (model.T, rule = demand1) 
        model.demand2Constraint = pyomo.Constraint (model.T, rule = demand2) 
        model.generatorCapacity11 = pyomo.Constraint (model.T, rule = generator_capacity_1_1) 
        model.generatorCapacity21 = pyomo.Constraint (model.T, rule = generator_capacity_2_1) 
        model.generatorCapacity12 = pyomo.Constraint (model.T, rule = generator_capacity_1_2) 
        model.generatorCapacity22 = pyomo.Constraint (model.T, rule = generator_capacity_2_2) 
        model.transmissionCapacityPositive = pyomo.Constraint (model.T,  rule = transmission_capacity_positive) 
        model.transmissionCapacityNegative = pyomo.Constraint(model.T, rule= transmission_capacity_negative) 

        #objective - solve 
        model.dual = pyomo.Suffix(direction=pyomo.Suffix.IMPORT)  
        result = solver.solve(model)
        print(model.objective())
        
        print(result.solver.status)
        print(result.solver.termination_condition)    
        print(model.display())


        row = {
        "s": s,
        "obj_value": pyomo.value(model.objective),
        "h_1_1": pyomo.value(model.h_1_1),
        "h_1_2": pyomo.value(model.h_1_2),
        "h_2_1": pyomo.value(model.h_2_1),
        "h_2_2": pyomo.value(model.h_2_2),
        "g_1_1_t[1]": pyomo.value(model.g_1_1_t[1]),
        "g_1_1_t[2]": pyomo.value(model.g_1_1_t[2]),
        "g_1_1_t[3]": pyomo.value(model.g_1_1_t[3]),
        "g_1_2_t[1]": pyomo.value(model.g_1_2_t[1]),
        "g_1_2_t[2]": pyomo.value(model.g_1_2_t[2]),
        "g_1_2_t[3]": pyomo.value(model.g_1_2_t[3]),
        "g_2_1_t[1]": pyomo.value(model.g_2_1_t[1]),
        "g_2_1_t[2]": pyomo.value(model.g_2_1_t[2]),
        "g_2_1_t[3]": pyomo.value(model.g_2_1_t[3]),
        "g_2_2_t[1]": pyomo.value(model.g_2_2_t[1]),
        "g_2_2_t[2]": pyomo.value(model.g_2_2_t[2]),
        "g_2_2_t[3]": pyomo.value(model.g_2_2_t[3]),
        "f_t [1]":     pyomo.value(model.f_t [1]),
        "f_t[2]":     pyomo.value(model.f_t[2]),
        "f_t[3]":     pyomo.value(model.f_t[3]),
        "F": pyomo.value(model.F)
        }
        
        # -----------------------------
        # Append to CSV
        # -----------------------------
        #with open(output_file, "a", newline="") as f:
        #    writer = csv.DictWriter(f, fieldnames=row.keys())
        #    if first_write:
        #        writer.writeheader()
        #        first_write = False
        #    writer.writerow(row)

        print("check KKT conditions")
        all_ok = True 

        #initiate list of lambda
        # no need dummy value because lambda1 is not dependent on t so no matching of dummy value is required   
        mu_pos = []
        mu_neg = []     
         
        #dummy value so that the index of lambda matches t 
        lambda1_list = [""]
        lambda2_list = [""]
        pi_1_1_list = [""]
        pi_1_2_list = [""]
        pi_2_1_list = [""]
        pi_2_2_list = [""]
        mu_pos = [""]
        mu_neg = [""]

        # DUAL                  
        for constr in model.component_objects(pyomo.Constraint, active=True):
            ok = True
            for idx in constr:
                c = constr[idx]
                val = pyomo.value(c.body)
                lb, ub = c.lower, c.upper

                # Get dual variable value (None if solver didn't return it)
                dual = model.dual.get(c, None)   
                #print(λ)         

                # primal 
                #print("primal")
                if lb is not None and val < lb :
                    print(f"[Primal Infeasibility] {c.name}: {val} < {lb}")
                    all_ok = False
                elif ub is not None and val > ub :
                    print(f"[Primal Infeasibility] {c.name}: {val} > {ub}")
                    all_ok = False
                    
                # ---- dual ----
                #print('dual')
                if dual is not None:
                    if lb is None and ub is not None:  # ≤ constraint # neg 
                        if dual > 0:
                            print(f"[Dual Infeasibility] {c.name}: dual = {dual} > 0")
                            all_ok = False
                    elif lb is not None and ub is None:  # ≥ constraint #non-neg 
                        if dual < 0:
                            print(f"[Dual Infeasibility] {c.name}: dual = {dual} < 0")
                            all_ok = False
                    elif lb == ub:  # equality constraint — no sign restriction
                        if lb == 0 and dual != 0: # when D = 0 
                            print (f"[Dual Infeasibility] {c.name}: dual = {dual}" ) 
                        elif lb > 0 and dual <= 0 : # when D > 0 
                            print(f"[Dual Infeasibility] {c.name}: dual = {dual} - no sign restriction")
                        pass

                if ok: 
                    if str (constr) == "demand1Constraint": 
                        lambda1_list.append(dual)                    
                    elif str(constr) == "demand2Constraint":
                        lambda2_list.append(dual)  
                    elif str(constr) == "generatorCapacity11": 
                        pi_1_1_list.append(dual)
                    elif str(constr) == "generatorCapacity21": 
                        pi_1_2_list.append(dual)
                    elif str(constr) == "generatorCapacity12": 
                        pi_2_1_list.append(dual)
                    elif str(constr) == "generatorCapacity22": 
                        pi_2_2_list.append(dual)
                    elif str(constr) == "transmissionCapacityPositive": 
                        mu_pos.append(dual)
                    elif str(constr) == "transmissionCapacityNegative":  
                        mu_neg.append(dual)                            

        if all_ok == False:
            print("Some KKT conditions failed")
        print("done with KKT checks\n")
        print("λ_1", lambda1_list)
        print("λ_2", lambda2_list)
        print("pi_1_1", pi_1_1_list)
        print("pi_1_2", pi_1_2_list)
        print("pi_2_1", pi_2_1_list)
        print("pi_2_2", pi_2_2_list)
        print("mu_pos", mu_pos)
        print("mu_neg", mu_neg)

        dual_dict ["λ_1"] = lambda1_list
        dual_dict ["λ_2"] = lambda2_list
        dual_dict ["pi_1_1"] = pi_1_1_list
        dual_dict ["pi_1_2"] = pi_1_2_list
        dual_dict ["pi_2_1"] = pi_2_1_list
        dual_dict ["pi_2_2"] = pi_2_2_list
        dual_dict["mu_pos"] = mu_pos
        dual_dict["mu_neg"] = mu_neg

        print("dual_dict", dual_dict)
        

s= 1
72.0
ok
optimal
Model unknown

  Variables:
    g_1_1_t : Size=3, Index=T
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 :   0.0 :  None : False : False : NonNegativeReals
    g_1_2_t : Size=3, Index=T
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   2.0 :  None : False : False : NonNegativeReals
          2 :     0 :  10.0 :  None : False : False : NonNegativeReals
          3 :     0 :  15.0 :  None : False : False : NonNegativeReals
    g_2_1_t : Size=3, Index=T
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 :   0.0 :  None : False : False : NonNegativeReals
    g_2_2_t : Size=3, Index=T
        K